In [1]:
import json
import datetime
import pandas as pd
import collections

In [2]:
def strp(date_string):
    return datetime.datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S.%f') 

def get_type(events, type):
    for e in events:
        type_object = e["type"]
        if type == type_object: return e

In [23]:
test = None

def single_deployment_analysis(filename):
    with open(filename, "r") as f:
        experiments = json.load(f)
    
    assert experiments[0]["single_deployment"]

    rows = []
    for e in experiments:
        data = e["data"]

        worker1_sd_creation = strp(get_type(data["selectivedeployment_events_worker1"], "created")["time"])

        worker2_sd_creation = strp(get_type(data["selectivedeployment_events_worker2"], "created")["time"])
        worker2_d_creation = strp(get_type(data["deployment_events"], "created")["time"])
        # worker2_ns_creation = strp(get_type(data[""], "created")["time"])

        test = data

        row = {**e}
        row.pop("data")

        # Here all the difference is are taken 
        row["worker2_sd_creation"] = (worker2_sd_creation - worker1_sd_creation).total_seconds()
        row["worker2_d_creation"] = (worker2_d_creation - worker1_sd_creation).total_seconds()

        rows.append(row)
    
    return pd.DataFrame(rows)

In [24]:
single_deployment_df = single_deployment_analysis("jsons/old/20230330_123916.json")
single_deployment_df

{'framework': 'edgenetfed',
 'iteration': 0,
 'num_iterations': 50,
 'pod_count': 1,
 'namespace': '71959dc7-5064-4ad1-b72a-a0cbe6a7df5c-test-34390dac',
 'federation_namespace': 'federated-15a381a9-c1cf-43a7-ae83-d6dac62e98d2',
 'single_deployment': True,
 'worker2_sd_creation': 1.916518,
 'worker2_d_creation': 1.996649}

In [11]:
set(single_deployment_df.test)

{3}

In [5]:
print("Replica count - Average Time for SD to Propagate from worker1 to worker2")
e = single_deployment_df.groupby("pod_count").selectivedeployment_from_worker1_to_worker2_selectivedeployment.mean()

for replicas, average_seconds in zip(e.index, e):
    print(f"{replicas} \t {average_seconds}s")

single_deployment_df.groupby("pod_count").selectivedeployment_from_worker1_to_worker2_selectivedeployment.describe()

Replica count - Average Time for SD to Propagate from worker1 to worker2
1 	 1.26511636s
5 	 1.35168084s
20 	 1.5385327599999998s


,count,mean,std,min,25%,50%,75%,max
pod_count,,,,,,,,
1,50.0,1.265116,0.343696,0.918914,1.065272,1.154291,1.318777,2.637117
5,50.0,1.351681,0.396793,1.007944,1.151960,1.235372,1.359362,2.936704
20,50.0,1.538533,0.684650,1.098807,1.225821,1.338717,1.604498,5.192611


In [110]:
def multi_deployment_analysis(filename):
    with open(filename, "r") as f:
        experiments = json.load(f)

    assert not experiments[0]["single_deployment"]

    rows = []
    for e in experiments:
        data = e["data"]

        rowdata = collections.defaultdict(list)

        for name_work1, name_work2 in zip(data["selectivedeployment_events_worker1"], data["selectivedeployment_events_worker2"]):
            sd_worker1 = strp(get_type(data["selectivedeployment_events_worker1"][name_work1], "created")["time"])
            sd_worker2 = strp(get_type(data["selectivedeployment_events_worker2"][name_work2], "created")["time"])
            rowdata["selectivedeployment_from_worker1_to_worker2_selectivedeployment"].append((sd_worker2 - sd_worker1).total_seconds())

        rowdata_df = pd.DataFrame(rowdata)

        row = {**e}
        row.pop("data")

        row["selectivedeployment_from_worker1_to_worker2_selectivedeployment_mean"] = rowdata_df.selectivedeployment_from_worker1_to_worker2_selectivedeployment.mean()

        rows.append(row)
        # break
    
    return pd.DataFrame(rows)

In [ ]:
multi_deployment_df = multi_deployment_analysis("jsons/old/20230330_091022.json")

In [111]:
print("Replica count - Average Time for SD to Propagate from worker1 to worker2")
e = multi_deployment_df.groupby("pod_count").selectivedeployment_from_worker1_to_worker2_selectivedeployment_mean.mean()

for replicas, average_seconds in zip(e.index, e):
    print(f"{replicas} \t {average_seconds}s")

multi_deployment_df.groupby("pod_count").selectivedeployment_from_worker1_to_worker2_selectivedeployment_mean.describe()

Replica count - Average Time for SD to Propagate from worker1 to worker2
1 	 1.25069492s
5 	 7.93875358s
20 	 28.675273403000002s


,count,mean,std,min,25%,50%,75%,max
pod_count,,,,,,,,
1,50.0,1.250695,0.411839,0.883943,0.993069,1.127334,1.333370,2.925436
5,50.0,7.938754,0.373540,7.371280,7.667160,7.831500,8.196874,8.918340
20,50.0,28.675273,1.130860,27.275419,27.783888,28.556859,29.332506,33.495867
